# Run functions from XHARPy from QCrBox

XHARPy is an external library to calculate atomic from factors from periodic PAW DFT calculations availablable on its own on [Github](https://github.com/Niolon/XHARPy). Two functionalities are currently exposed within QCrBox: calculation of atomic form factors and full HAR refinement.

## Import and setting up folders / paths

In [1]:
import shutil

from qcrbox_wrapper import QCrBoxPathHelper, QCrBoxWrapper

We create an example folder to try out this functionality.


Using the dotenv package makes things more convenient as we can read the environment variables from the .env.dev file in the QCrBox directory. If you want to run with python core packages only, use the `__init__` method instead by defining the path to the shared directory explicitely in `path_to_shared_dir` and replacing the next four lines with:


```python
pathhelper = QCrBoxPathHelper(
    path_to_shared_dir,
    'examples_xharpy'
)
```

In [2]:
pathhelper = QCrBoxPathHelper.from_dotenv(".env.dev", "examples_xharpy")

path_local = pathhelper.local_path
path_qcrbox = pathhelper.qcrbox_path

## Connecting to QCrBox
We can connect to the QCrBox Inventory via python after we have started everything with qcb up. Sometimes the server takes a while so you might need to retry if it initially refuses connection. This should not take more than 30 seconds after your console output says that everything has started.

In [3]:
qcrbox = QCrBoxWrapper.from_server_addr("127.0.0.1", 11000)

In [4]:
qcrbox.application_dict

2024-08-22T23:07:28.339795Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}
2024-08-22T23:07:28.340955Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


{'QCrBoxTools': <QCrBoxTools>, 'XHARPy-GPAW': <XHARPy-GPAW>}

In [5]:
xharpy = qcrbox.application_dict["XHARPy-GPAW"]

2024-08-22T23:07:28.353833Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}
2024-08-22T23:07:28.355692Z [debug    ] TODO: implement proper construction and validation of gui_url extra={}


In [6]:
help(xharpy)

Help on QCrBoxApplication in module qcrbox_wrapper.qcrbox_application object:

class QCrBoxApplication(builtins.object)
 |  QCrBoxApplication(application_spec: pyqcrbox.sql_models_NEW_v2.application_spec.ApplicationSpecWithCommands, wrapper_parent: 'QCrBoxWrapper') -> None
 |
 |  Represents an application in QCrBox packaged in its own container.
 |
 |  Methods defined here:
 |
 |  __init__(self, application_spec: pyqcrbox.sql_models_NEW_v2.application_spec.ApplicationSpecWithCommands, wrapper_parent: 'QCrBoxWrapper') -> None
 |      Initializes the QCrBoxApplication instance.
 |
 |      Parameters
 |      ----------
 |      application_spec: sql_models_NEW_v2.ApplicationSpecWithCommands
 |          The application spec as returned by the API endpoint `/applications`.
 |
 |  __repr__(self) -> str
 |      Return repr(self).
 |
 |  interactive_session(self, **kwargs)
 |
 |  ----------------------------------------------------------------------
 |  Readonly properties defined here:
 |
 |  

## Calculating aspheric atomic form factors and writing them into a tsc file

A `tsc` file is the current standard way of exchanging non-spheric atomic form factors. Here we can calculate the tsc for the atoms within our asymmetric unit for the given crystal structure using GPAW. Functionals are any functionals that GPAW can use. A smaller grid spacing means more grid points *i.e.* a larger basis set in our calculation.


In [7]:
# create a new folder for this example
folder_aff = path_local / "run_tsc_calc"
folder_aff.mkdir(exist_ok=True)

# copy file
shutil.copy("./input_files/input.cif", folder_aff / "input.cif")

# start command
calc = xharpy.atom_form_fact_gpaw(
    input_cif_path=path_qcrbox / "run_tsc_calc" / "input.cif",
    output_tsc_path=path_qcrbox / "run_tsc_calc" / "affs.tsc",
    functional="PBE",
    gridspacing=0.20,
)

# wait for command to finish
calc.wait_while_running(1.0)

## Running a Hirshfeld Atom Refinement
XHARPy can also refine a structure directly. As the XHARPy refinement is slower than that of Olex2 this should probably be seen as an example more than a direct implementation. Functionals are any functionals that GPAW can use. A smaller grid spacing means more grid points *i.e.* a larger basis set in our calculation.

In [8]:
# create a new folder for this example
folder_har = path_local / "run_har"
folder_har.mkdir(exist_ok=True)

# copy file
shutil.copy("./input_files/input.cif", folder_har / "input.cif")

# start command
calc2 = xharpy.ha_refine(
    input_cif_path=path_qcrbox / "run_har" / "input.cif",
    output_cif_path=path_qcrbox / "run_har" / "output.cif",
    functional="PBE",
    gridspacing=0.20,
)

# wait for command to finish
print(calc2.status)
calc2.wait_while_running(1.0)
print(calc2.status)

submitted


UnsuccessfulCalculationError: Calculation with id qcrbox_calc_0x5b0d4e6f94874452a8be598b3715822b does has status 'failed'.

Potential error message:
No error message available

Command stdout:
Retrieval of STDOUT not implemented yet for PythonCallableCalculation

Command stderr:
Retrieval of STDERR not implemented yet for PythonCallableCalculation